### Dependencies

In [159]:
import sys
!{sys.executable} -m pip install xgboost
!{sys.executable} -m pip install tqdm
import numpy as np
import pandas as pd
from tqdm import tqdm
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


### Process Train Set

In [160]:
train_set = pd.read_csv("data/train.csv")
male_sex_df = pd.get_dummies(train_set["Sex"])["male"]
train_set.drop(columns=["PassengerId", "Name", "Cabin", "Ticket", "Sex", "Embarked"], inplace=True)
train_set = pd.concat([train_set, male_sex_df], axis=1)
train_set.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


### Train Model

In [168]:
x, y = train_set.drop(columns=["Survived"]), train_set["Survived"]
best_model, best_acc = None, -1

for i in tqdm(range(100)):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    x_train.head()
    train = xgb.DMatrix(x_train, label=y_train)
    param = {
        "max_depth": 3,
        "eta": 0.05,
        "objective": "multi:softprob",
        "num_class": 3
    }
    epochs = 25

    model = xgb.train(param, train, epochs)
    predictions = np.argmax(model.predict(xgb.DMatrix(x)), axis=1)
    acc = accuracy_score(predictions, y)
    if acc >= best_acc:
        best_acc = acc
        best_model = model
        
best_model.save_model("model.json") 
print("Best model found with an accuracy of: " + str(best_acc))
print("Model created and saved!!!")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 42.62it/s]

Best model found with an accuracy of: 0.8473625140291807
Model created and saved!!!


### Process Test Set

In [169]:
prediction_set = pd.read_csv("data/test.csv")
male_sex_df = pd.get_dummies(prediction_set["Sex"])["male"]
predictions_series = prediction_set["PassengerId"]
prediction_set.drop(columns=["PassengerId", "Name", "Cabin", "Ticket", "Sex", "Embarked"], inplace=True)
prediction_set = pd.concat([prediction_set, male_sex_df], axis=1)
prediction_set.head()

,Pclass,Age,SibSp,Parch,Fare,male
0,3,34.5,0,0,7.8292,1
1,3,47.0,1,0,7.0000,0
2,2,62.0,0,0,9.6875,1
3,3,27.0,0,0,8.6625,1
4,3,22.0,1,1,12.2875,0


#### Run Model

In [170]:
model = xgb.Booster()
model.load_model("model.json")
survived = model.predict(xgb.DMatrix(prediction_set))
prediction_list = np.argmax(survived, axis=1).tolist()
predictions = pd.DataFrame(predictions_series)
predictions["Survived"] = prediction_list
predictions.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [171]:
predictions.to_csv("predictions.csv", index=False)